In [14]:
import requests
import pandas as pd
import plotly.graph_objs as go
from datetime import datetime

# --- PARAMETERS ---
LATITUDE = 52.52
LONGITUDE = 13.41
TIMEZONE = "Europe/London"
PAST_DAYS = 3

# --- FETCH DATA FROM OPEN-METEO API (PAST + FUTURE) ---
def fetch_combined_weather(lat, lon, timezone, past_days):
    url = (
        f"https://api.open-meteo.com/v1/forecast"
        f"?latitude={lat}&longitude={lon}"
        f"&hourly=temperature_2m,weather_code"
        f"&timezone={timezone}"
        f"&past_days={past_days}"
    )

    resp = requests.get(url)
    data = resp.json()

    df = pd.DataFrame({
        "time": pd.to_datetime(data["hourly"]["time"]),
        "temperature_2m": data["hourly"]["temperature_2m"],
        "weather_code": data["hourly"]["weather_code"]
    })

    # --- FIX: Make comparison with tz-naive now ---
    now = pd.Timestamp.now(tz=None)
    df["is_forecast"] = df["time"] > now

    return df

# --- PLOT COMBINED GRAPH WITH COLORED SEGMENTS ---
def plot_combined_weather_colored(df):
    fig = go.Figure()

    # Historical temperature
    fig.add_trace(go.Scatter(
        x=df[df["is_forecast"] == False]["time"],
        y=df[df["is_forecast"] == False]["temperature_2m"],
        mode='lines+markers',
        name='Past Temp (°C)',
        yaxis='y1',
        line=dict(color='firebrick')
    ))

    # Forecast temperature
    fig.add_trace(go.Scatter(
        x=df[df["is_forecast"] == True]["time"],
        y=df[df["is_forecast"] == True]["temperature_2m"],
        mode='lines+markers',
        name='Forecast Temp (°C)',
        yaxis='y1',
        line=dict(color='darkgreen')
    ))

    # Historical weather code
    fig.add_trace(go.Scatter(
        x=df[df["is_forecast"] == False]["time"],
        y=df[df["is_forecast"] == False]["weather_code"],
        mode='lines+markers',
        name='Past Weather Code',
        yaxis='y2',
        line=dict(color='royalblue')
    ))

    # Forecast weather code
    fig.add_trace(go.Scatter(
        x=df[df["is_forecast"] == True]["time"],
        y=df[df["is_forecast"] == True]["weather_code"],
        mode='lines+markers',
        name='Forecast Weather Code',
        yaxis='y2',
        line=dict(color='orange')
    ))

    # Layout with dual y-axes
    fig.update_layout(
        title="Open-Meteo: Past & Forecast Weather (Hourly)",
        xaxis=dict(title="Time"),
        yaxis=dict(
            title="Temperature (°C)",
            titlefont=dict(color="firebrick"),
            tickfont=dict(color="firebrick")
        ),
        yaxis2=dict(
            title="Weather Code",
            titlefont=dict(color="royalblue"),
            tickfont=dict(color="royalblue"),
            anchor="x",
            overlaying="y",
            side="right"
        ),
        height=700,
        width=1500,
        legend=dict(
    orientation="v",
    x=1.02,  # Move to the right of the chart
    y=1,
    bgcolor="rgba(0,0,0,0)",  # Transparent background
    borderwidth=0
),

        margin=dict(l=60, r=60, t=60, b=60)
    )

    fig.show()

# --- RUN PIPELINE ---
df_weather = fetch_combined_weather(LATITUDE, LONGITUDE, TIMEZONE, PAST_DAYS)
plot_combined_weather_colored(df_weather)


#Code to get the csv and html file as a output of the stamps

In [17]:
import requests
import pandas as pd
import plotly.graph_objs as go
from datetime import datetime
import time
import os

# --- CONFIGURATION ---
LATITUDE = 52.52
LONGITUDE = 13.41
TIMEZONE = "Europe/London"
PAST_DAYS = 3
RUN_INTERVAL_SECONDS = 3600  # 1 hour

OUTPUT_DIR = "output"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# --- DATA FETCHING ---
def fetch_combined_weather(lat, lon, timezone, past_days):
    url = (
        f"https://api.open-meteo.com/v1/forecast"
        f"?latitude={lat}&longitude={lon}"
        f"&hourly=temperature_2m,weather_code"
        f"&timezone={timezone}"
        f"&past_days={past_days}"
    )
    resp = requests.get(url)
    data = resp.json()

    df = pd.DataFrame({
        "time": pd.to_datetime(data["hourly"]["time"]),
        "temperature_2m": data["hourly"]["temperature_2m"],
        "weather_code": data["hourly"]["weather_code"]
    })

    now = pd.Timestamp.now(tz=None)
    df["is_forecast"] = df["time"] > now
    return df, now

# --- PLOTTING ---
def plot_combined_weather_colored(df, last_updated, output_path=None):
    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=df[~df["is_forecast"]]["time"],
        y=df[~df["is_forecast"]]["temperature_2m"],
        mode='lines+markers',
        name='Past Temp (°C)',
        yaxis='y1',
        line=dict(color='firebrick')
    ))

    fig.add_trace(go.Scatter(
        x=df[df["is_forecast"]]["time"],
        y=df[df["is_forecast"]]["temperature_2m"],
        mode='lines+markers',
        name='Forecast Temp (°C)',
        yaxis='y1',
        line=dict(color='darkgreen')
    ))

    fig.add_trace(go.Scatter(
        x=df[~df["is_forecast"]]["time"],
        y=df[~df["is_forecast"]]["weather_code"],
        mode='lines+markers',
        name='Past Weather Code',
        yaxis='y2',
        line=dict(color='royalblue')
    ))

    fig.add_trace(go.Scatter(
        x=df[df["is_forecast"]]["time"],
        y=df[df["is_forecast"]]["weather_code"],
        mode='lines+markers',
        name='Forecast Weather Code',
        yaxis='y2',
        line=dict(color='orange')
    ))

    fig.update_layout(
        title=f"Automated Weather Analytics Dashboard<br><sub>Last updated: {last_updated.strftime('%Y-%m-%d %H:%M:%S')}</sub>",
        xaxis=dict(title="Time"),
        yaxis=dict(
            title="Temperature (°C)",
            titlefont=dict(color="firebrick"),
            tickfont=dict(color="firebrick")
        ),
        yaxis2=dict(
            title="Weather Code",
            titlefont=dict(color="royalblue"),
            tickfont=dict(color="royalblue"),
            anchor="x",
            overlaying="y",
            side="right"
        ),
        height=600,
        width=1100,
        margin=dict(l=60, r=160, t=80, b=60),
        legend=dict(
            orientation="v",
            x=1.02,
            y=1,
            bgcolor="rgba(0,0,0,0)",
            borderwidth=0
        )
    )

    if output_path:
        fig.write_html(output_path)
    else:
        fig.show()

# --- SAVE DATA ---
def save_data(df, output_path):
    df.to_csv(output_path, index=False)

# --- MAIN ---
def run_pipeline():
    df, last_updated = fetch_combined_weather(LATITUDE, LONGITUDE, TIMEZONE, PAST_DAYS)

    timestamp_str = last_updated.strftime("%Y%m%d_%H%M%S")
    csv_path = os.path.join(OUTPUT_DIR, f"weather_data_{timestamp_str}.csv")
    html_path = os.path.join(OUTPUT_DIR, f"weather_plot_{timestamp_str}.html")

    save_data(df, csv_path)
    plot_combined_weather_colored(df, last_updated, output_path=html_path)

    print(f"[{datetime.now()}] Pipeline run completed.")
    print(f"Data saved to: {csv_path}")
    print(f"Plot saved to: {html_path}")

def main_loop():
    while True:
        try:
            run_pipeline()
        except Exception as e:
            print(f"[{datetime.now()}] Error during pipeline run: {e}")
        time.sleep(RUN_INTERVAL_SECONDS)

if __name__ == "__main__":
    main_loop()


[2025-06-19 21:56:00.310430] Pipeline run completed.
Data saved to: output/weather_data_20250619_215600.csv
Plot saved to: output/weather_plot_20250619_215600.html


KeyboardInterrupt: 